In [1]:
from io import BytesIO
import subprocess
import pandas as pd

from sklearn import preprocessing

In [ ]:
def final_score(sl, sm, sw, wl=1, wm=1, ww=1):
    weighted_geometric_mean = ((sl**wl)*(sm**wm))**(1/(wl + wm))
    scaling_factor = ww*(sw-sm)/(sm*(wl+wm+ww)) + 1
    return weighted_geometric_mean*scaling_factor

In [ ]:
def fn(seqfile):
    cmd = f"filtlong --min_mean_q 1 --verbose {seqfile} 2>&1 >/dev/null | grep -P 'length = \d+' | sed 's/\s\+length = //' | sed 's/\s\+\w\+ quality = /\t/g'"
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True, check=True)
    
    df = pd.read_csv(BytesIO(p.stdout), sep='\t', names=['length', 'mean_quality', 'window_quality'])
    df['length_score'] = df['length'].apply(lambda x: (1 + (-5000/(x + 5000)))*100)
    
    z_score = preprocessing.scale(df['mean_quality'])
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))
    min_max_data = min_max_scaler.fit_transform(z_score.reshape(-1, 1))
    
    df['mean_quality_score'] = min_max_data.flatten()
    df['window_quality_score'] = df['window_quality']/df['mean_quality']*df['mean_quality_score']
    df['reads_score'] = df.apply(lambda x: final_score(
        x['length_score'], x['mean_quality_score'], x['window_quality_score']
    ), axis=1)
    return df

In [2]:
n = 128
[10**(q / -10) for q in range(n+1)][13]*3000000

150356.17008818165

In [247]:
import numpy as np
from scipy.stats import ttest_ind_from_stats

In [252]:
def t_test(group1, group2):
    mean1 = np.mean(group1)
    mean2 = np.mean(group2)
    std1 = np.std(group1)
    std2 = np.std(group2)
    nobs1 = len(group1)
    nobs2 = len(group2)
    
    modified_std1 = np.sqrt(np.float32(nobs1)/
                    np.float32(nobs1-1)) * std1
    modified_std2 = np.sqrt(np.float32(nobs2)/
                    np.float32(nobs2-1)) * std2
    result = ttest_ind_from_stats(
        mean1=mean1, std1=modified_std1, nobs1=nobs1,
        mean2=mean2, std2=modified_std2, nobs2=nobs2)
    return result

In [256]:
data_1 = nanoget.extraction_functions.process_fastq_minimal('/media/NGS/Nanopore_1/20181127_R18-0877/fastq/R18-0877.fastq')
data_2 = nanoget.extraction_functions.process_fastq_minimal('/media/NGS/Nanopore_1/20210129_Listera/fastq/R20.0160.fastq')

In [257]:
t_test(data_1['lengths'].values, data_2['lengths'].values)

Ttest_indResult(statistic=-36.8614591448439, pvalue=3.183280742105188e-297)